<a href="https://colab.research.google.com/github/iciromaco/Trial/blob/master/%E6%9B%B8%E5%89%B2%E5%8A%B9%E6%9E%9C%E3%82%B7%E3%83%9F%E3%83%A5%E3%83%AC%E3%83%BC%E3%82%BF%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/iciromaco/Trial/blob/master/%E6%9B%B8%E5%89%B2%E5%8A%B9%E6%9E%9C%E3%82%B7%E3%83%9F%E3%83%A5%E3%83%AC%E3%83%BC%E3%82%BF%E3%83%BC.ipynb)


In [ ]:
# @title 【ポール設置版】書割効果シミュレーター
# @markdown **変更点:**
# @markdown - **ポール設置:** 球体を高さ 2.5m の位置に持ち上げ、細いポールで支えました。
# @markdown - **効果:** 地面から切り離されたことで、特に「模様なし（Alpha=0）」や「霞み（Fog）」が強い時に、**まるで道路標識（円盤）のように見えます。**
# @markdown
# @markdown **おすすめの観察設定:**
# @markdown 1. **Distance = 50m**
# @markdown 2. **Fog Density = 0.3** (少し霞ませる)
# @markdown 3. **Pattern Alpha = 0.0** (模様を消す！)
# @markdown    - **結果:** 陰影だけの球がポールに乗っていると、立体感が消え失せ、**「グレーの円盤看板」**に見えませんか？
# @markdown 4. **Elongation UP:** そこで奥行きを伸ばすと、急に「アメ玉」のような立体感が戻ります。

import numpy as np
import cv2
import ipywidgets as widgets
from IPython.display import display, clear_output

def render_pole_scene(h, w, target_dist, eye_m, elongation, pat_alpha, fog_density):
    # ==========================================
    # 1. カメラ設定
    # ==========================================
    sensor_h_mm = 24.0
    target_radius = 1.0
    fill_rate = 0.3
    f_mm = (target_dist * 1000) * (sensor_h_mm * fill_rate) / (target_radius * 2 * 1000)
    f_px = f_mm * (h / sensor_h_mm)

    cx = w / 2.0
    cy = h / 2.0
    cam_h = 1.6 # アイレベル

    # ==========================================
    # 2. オブジェクト定義
    # ==========================================
    # ★修正: 高さを上げてポールに乗せる
    sphere_y = 2.5 # 地上2.5m中心

    # (1) ターゲット球 (赤)
    sphere_target1 = {
        'center': np.array([1.9, sphere_y, target_dist]),
        'radius': 0.7,
        'elongation': elongation,
        'type': 'checker',
        'color': np.array([50, 50, 255])
    }

    # (2) 参照球 (青)
    sphere_ref1 = {
        'center': np.array([-1.0, sphere_y-1.0, target_dist - 2.0]),
        'radius': 0.5,
        'elongation': 1.0,
        'type': 'geo',
        'color': np.array([255, 100, 50])
    }

    # (3) ターゲット球 (赤2)
    sphere_target2 = {
        'center': np.array([0.9, sphere_y -1.0, target_dist + 60.0]),
        'radius': 0.6,
        'elongation': elongation,
        'type': 'checker',
        'color': np.array([50, 50, 255])
    }

    # (4) 参照球 (青2)
    sphere_ref2 = {
        'center': np.array([-2.0, sphere_y, target_dist + 58.0]),
        'radius': 0.7,
        'elongation': 1.0,
        'type': 'geo',
        'color': np.array([255, 100, 50])
    }

    objects = [sphere_target1, sphere_ref1, sphere_target2, sphere_ref2]

    # ポール設定
    pole_radius = 0.2
    pole_color = np.array([100, 100, 100]) # グレー

    wall_x = 3.5
    wall_h = 8.0 # 壁も少し高く

    # ==========================================
    # 3. ヘルパー関数
    # ==========================================
    def create_lookat_matrix(origin, target, up=np.array([0,1,0])):
        fwd = target - origin
        fwd_len = np.linalg.norm(fwd)
        if fwd_len < 1e-6: return np.eye(3)
        fwd /= fwd_len
        right = np.cross(up, fwd)
        right_len = np.linalg.norm(right)
        if right_len < 1e-6: right = np.array([1,0,0])
        else: right /= right_len
        true_up = np.cross(fwd, right)
        mat = np.stack([right, true_up, fwd])
        return mat

    # ==========================================
    # 4. レンダリング
    # ==========================================
    y_idx, x_idx = np.indices((h, w))

    def trace_eye(eye_offset):
        cam_pos = np.array([eye_offset, cam_h, 0.0])

        # レイ生成
        ray_x = (x_idx - cx)
        ray_y = -(y_idx - cy)
        ray_z = np.full((h, w), f_px, dtype=np.float32)

        norm = np.sqrt(ray_x**2 + ray_y**2 + ray_z**2)
        norm[norm==0] = 1.0
        dx = ray_x / norm
        dy = ray_y / norm
        dz = ray_z / norm

        z_buffer = np.full((h, w), np.inf, dtype=np.float32)

        # 背景 (空)
        sky_clear_b, sky_clear_g, sky_clear_r = 250*0.9, 250*0.9, 255*0.8
        sky_fog_b, sky_fog_g, sky_fog_r = 180, 200, 215

        # フォグ密度に応じて背景色を変化
        bg_col = np.zeros(3, dtype=np.float32)
        bg_col[0] = sky_clear_b * (1 - fog_density*0.5) + sky_fog_b * (fog_density*0.5)
        bg_col[1] = sky_clear_g * (1 - fog_density*0.5) + sky_fog_g * (fog_density*0.5)
        bg_col[2] = sky_clear_r * (1 - fog_density*0.5) + sky_fog_r * (fog_density*0.5)

        sky_fac = np.clip(1.0 - y_idx/(h*0.6), 0.0, 1.0)
        img = np.zeros((h, w, 3), dtype=np.float32)
        img[:,:,0] = sky_fac * sky_clear_b + (1-sky_fac)*bg_col[0]
        img[:,:,1] = sky_fac * sky_clear_g + (1-sky_fac)*bg_col[1]
        img[:,:,2] = sky_fac * sky_clear_r + (1-sky_fac)*bg_col[2]

        # フォグ関数
        def apply_contrast_fog(colors, dists):
            factor = fog_density * 0.025
            fade_amount = 1.0 - np.exp(-dists * factor)
            fade_amount = np.clip(fade_amount[:, np.newaxis], 0, 1)
            return colors * (1 - fade_amount) + bg_col * fade_amount

        # --- A. 地面 ---
        mask_dy = dy < -0.001
        if np.any(mask_dy):
            tr = -cam_pos[1] / dy[mask_dy]
            mask_valid = tr > 0
            if np.any(mask_valid):
                tr = tr[mask_valid]
                temp_y, temp_x = np.where(mask_dy)
                fy = temp_y[mask_valid]; fx = temp_x[mask_valid]

                Px = cam_pos[0] + tr * dx[fy, fx]
                Pz = cam_pos[2] + tr * dz[fy, fx]

                col_road = np.ones((len(tr), 3)) * 60
                is_line = (np.abs(Px) < 0.15) & ((Pz % 6.0) < 3.0)
                col_road[is_line] = [220, 220, 220]

                # ポールの影 (簡易) - 真下付近を暗く
                for obj in objects:
                    C = obj['center']
                    dist_to_pole = np.sqrt((Px - C[0])**2 + (Pz - C[2])**2)
                    shadow = (dist_to_pole < 0.8)
                    col_road[shadow] *= 0.6

                img[fy, fx] = apply_contrast_fog(col_road, tr)
                z_buffer[fy, fx] = tr

        # --- B. 壁 ---
        for wx in [-wall_x, wall_x]:
            with np.errstate(divide='ignore', invalid='ignore'):
                t_wall = (wx - cam_pos[0]) / dx
            valid_t = np.isfinite(t_wall) & (t_wall > 0)
            Py_w = np.full((h, w), -999.0, dtype=np.float32)
            Py_w[valid_t] = cam_pos[1] + t_wall[valid_t] * dy[valid_t]
            mask_w = valid_t & (Py_w >= 0) & (Py_w <= wall_h) & (t_wall < z_buffer)

            if np.any(mask_w):
                tw = t_wall[mask_w]
                col_w = np.ones((len(tw), 3)) * 120

                # 窓模様
                Pz_w = cam_pos[2] + tw * dz[mask_w]
                Py_v = Py_w[mask_w]
                win = ((Pz_w % 4.0) < 2.0) & ((Py_v % 3.0) > 1.0)
                col_w[win] = [60, 60, 90]

                img[mask_w] = apply_contrast_fog(col_w, tw)
                z_buffer[mask_w] = tw

        # --- C. ポール (円柱) ---
        for obj in objects:
            C = obj['center']
            # ポールの位置: (Cx, 0, Cz) から (Cx, Cy, Cz) まで
            # 円柱判定: (Px - Cx)^2 + (Pz - Cz)^2 = r^2
            Oc_x = cam_pos[0] - C[0]
            Oc_z = cam_pos[2] - C[2]

            # A*t^2 + B*t + C = 0 (XZ平面)
            # dx, dz成分のみ使用
            A_cyl = dx**2 + dz**2
            B_cyl = 2 * (Oc_x*dx + Oc_z*dz)
            C_cyl = Oc_x**2 + Oc_z**2 - pole_radius**2

            Delta = B_cyl**2 - 4*A_cyl*C_cyl
            mask_p = Delta >= 0

            if np.any(mask_p):
                sqrt_d = np.sqrt(Delta[mask_p])
                t_p = (-B_cyl[mask_p] - sqrt_d) / (2 * A_cyl[mask_p])

                # 高さチェック
                hy, hx = np.where(mask_p)
                Py_p = cam_pos[1] + t_p * dy[hy, hx]

                valid_p = (t_p > 0) & (Py_p >= 0) & (Py_p <= C[1]) # 地面から球中心まで
                valid_p &= (t_p < z_buffer[hy, hx])

                if np.any(valid_p):
                    fy = hy[valid_p]; fx = hx[valid_p]; ft = t_p[valid_p]

                    # シェーディング (円柱法線)
                    Px = cam_pos[0] + ft * dx[fy, fx]
                    Pz = cam_pos[2] + ft * dz[fy, fx]
                    Nx = (Px - C[0]) / pole_radius
                    Nz = (Pz - C[2]) / pole_radius

                    Lx, Lz = 0.6, -0.5 # 光源
                    diff = np.clip(Nx*Lx + Nz*Lz, 0.2, 1.0)

                    col_p = pole_color * diff[:,np.newaxis]
                    img[fy, fx] = apply_contrast_fog(col_p, ft)
                    z_buffer[fy, fx] = ft

        # --- D. 球体 ---
        for obj in objects:
            C = obj['center']
            R = obj['radius']
            Elong = obj['elongation']

            R_mat = create_lookat_matrix(cam_pos, C)
            Oc = cam_pos - C
            L_Ox = R_mat[0,0]*Oc[0] + R_mat[0,1]*Oc[1] + R_mat[0,2]*Oc[2]
            L_Oy = R_mat[1,0]*Oc[0] + R_mat[1,1]*Oc[1] + R_mat[1,2]*Oc[2]
            L_Oz = R_mat[2,0]*Oc[0] + R_mat[2,1]*Oc[1] + R_mat[2,2]*Oc[2]
            L_Dx = R_mat[0,0]*dx + R_mat[0,1]*dy + R_mat[0,2]*dz
            L_Dy = R_mat[1,0]*dx + R_mat[1,1]*dy + R_mat[1,2]*dz
            L_Dz = R_mat[2,0]*dx + R_mat[2,1]*dy + R_mat[2,2]*dz

            Rx = R; Ry = R; Rz = R * Elong
            O_sx = L_Ox/Rx; O_sy = L_Oy/Ry; O_sz = L_Oz/Rz
            D_sx = L_Dx/Rx; D_sy = L_Dy/Ry; D_sz = L_Dz/Rz

            A = D_sx**2 + D_sy**2 + D_sz**2
            B = 2 * (O_sx*D_sx + O_sy*D_sy + O_sz*D_sz)
            C_val = O_sx**2 + O_sy**2 + O_sz**2 - 1.0
            Delta = B**2 - 4*A*C_val

            hit_mask = (Delta >= 0)
            if np.any(hit_mask):
                sqrt_d = np.sqrt(Delta[hit_mask])
                t_hit = (-B[hit_mask] - sqrt_d) / (2 * A[hit_mask])
                hy, hx = np.where(hit_mask)
                current_z = z_buffer[hy, hx]
                valid_z = (t_hit > 0) & (t_hit < current_z)

                if np.any(valid_z):
                    fy = hy[valid_z]; fx = hx[valid_z]; ft = t_hit[valid_z]

                    L_Px = L_Ox + ft * L_Dx[fy, fx]
                    L_Py = L_Oy + ft * L_Dy[fy, fx]
                    L_Pz = L_Oz + ft * L_Dz[fy, fx]

                    N_loc_x = L_Px / R
                    N_loc_y = L_Py / R
                    N_loc_z = (L_Pz / Elong) / R

                    Nx = R_mat[0,0]*N_loc_x + R_mat[1,0]*N_loc_y + R_mat[2,0]*N_loc_z
                    Ny = R_mat[0,1]*N_loc_x + R_mat[1,1]*N_loc_y + R_mat[2,1]*N_loc_z
                    Nz = R_mat[0,2]*N_loc_x + R_mat[1,2]*N_loc_y + R_mat[2,2]*N_loc_z
                    len_n = np.sqrt(Nx**2 + Ny**2 + Nz**2)
                    Nx/=len_n; Ny/=len_n; Nz/=len_n

                    Lx, Ly, Lz = 0.6, 0.7, -0.4
                    L_len = np.sqrt(Lx**2+Ly**2+Lz**2)
                    diff = np.maximum(0, (Nx*Lx + Ny*Ly + Nz*Lz)/L_len)
                    shade = 0.3 + 0.7 * diff

                    if obj['type'] == 'checker':
                        rot_tilt = 0.4
                        Ny_r = Ny*np.cos(rot_tilt) - Nz*np.sin(rot_tilt)
                        Nz_r = Ny*np.sin(rot_tilt) + Nz*np.cos(rot_tilt)
                        u = np.arctan2(Nx, -Nz_r)
                        v = np.arcsin(np.clip(Ny_r, -1, 1))
                        freq = 8.0
                        pat = (np.floor((u+np.pi)/(2*np.pi)*freq*2) + np.floor((v+np.pi/2)/np.pi*freq)) % 2 == 0
                        col_pat = np.zeros((len(ft), 3))
                        col_pat[pat] = [240, 240, 240]
                        col_pat[~pat] = obj['color']
                        col_solid = np.tile(obj['color'], (len(ft), 1))
                    else:
                        rx = np.radians(45); ry = np.radians(-30)
                        Nx_t = Nx*np.cos(ry) - Nz*np.sin(ry)
                        Nz_t = Nx*np.sin(ry) + Nz*np.cos(ry)
                        Ny_t = Ny*np.cos(rx) - Nz_t*np.sin(rx)
                        Nz_t = Ny*np.sin(rx) + Nz_t*np.cos(rx)
                        u = np.arctan2(Nx_t, -Nz_t)
                        v = np.arcsin(np.clip(Ny_t, -1, 1))
                        step = 15
                        is_line = (np.abs(np.degrees(u).astype(int)) % step < 2) | (np.abs(np.degrees(v).astype(int)) % step < 2)
                        col_pat = np.tile(obj['color'], (len(ft), 1))
                        col_pat[is_line] = [255, 255, 255]
                        col_solid = np.tile(obj['color'], (len(ft), 1))

                    base_col = col_pat * pat_alpha + col_solid * (1.0 - pat_alpha)
                    final_col = base_col * shade[:,np.newaxis]
                    img[fy, fx] = apply_contrast_fog(final_col, ft)
                    z_buffer[fy, fx] = ft

        return img.astype(np.uint8), f_mm

    offset = eye_m / 2.0
    img_L, f_val = trace_eye(-offset)
    img_R, _ = trace_eye(offset)

    return img_L, img_R, f_val

# --- UI ---
style = {'description_width': 'initial'}
layout = widgets.Layout(width='95%')

s_dist = widgets.FloatSlider(value=50.0, min=5.0, max=300.0, step=5.0, description='奥の球の距離 (m):', style=style, layout=layout)
s_fog = widgets.FloatSlider(value=0.2, min=0.0, max=1.0, step=0.1, description='空気の霞み (Fog Density):', style=style, layout=layout)
s_pat = widgets.FloatSlider(value=0.2, min=0.0, max=1.0, step=0.1, description='模様の濃さ (Pattern Alpha):', style=style, layout=layout)
s_elong = widgets.FloatSlider(value=1.0, min=1.0, max=20.0, step=1.0, description='奥行き伸長 (補正):', style=style, layout=layout)
s_eye = widgets.FloatSlider(value=0.065, min=0.0, max=0.2, step=0.005, description='瞳孔間距離 (m):', style=style, layout=layout)
d_mode = widgets.Dropdown(options=['交差法 (Cross)', '平行法 (Parallel)'], value='平行法 (Parallel)', description='観察モード:', style=style)

out = widgets.Output()

def update(_):
    dist = s_dist.value
    elong = s_elong.value
    eye = s_eye.value
    fog = s_fog.value
    pat = s_pat.value
    mode = d_mode.value

    img_L, img_R, f_val = render_pole_scene(400, 400, dist, eye, elong, pat, fog)

    def decorate(img, t):
        cv2.rectangle(img, (0,0), (399,399), (255,255,255), 3)
        cv2.putText(img, t, (20,50), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,255,255), 3)
        return img

    if 'Cross' in mode:
        comb = np.hstack((decorate(img_R, "R"), decorate(img_L, "L")))
    else:
        comb = np.hstack((decorate(img_L, "L"), decorate(img_R, "R")))

    _, enc = cv2.imencode('.png', comb)

    out.clear_output(wait=True)
    with out:
        display(widgets.Image(value=enc.tobytes(), format='png'))
        print(f"距離: {dist}m | 焦点距離: {f_val:.0f}mm")
        if pat < 0.1:
             print("★ 模様OFF: ポール上の球が「無地の看板」に見えませんか？")
        elif fog > 0.3 and dist > 30:
             print("★ 書割効果: ポールによって接地感が消え、平板化が強調されています。")

s_dist.observe(update, names='value')
s_elong.observe(update, names='value')
s_eye.observe(update, names='value')
s_pat.observe(update, names='value')
s_fog.observe(update, names='value')
d_mode.observe(update, names='value')

update(None)

display(widgets.VBox([
    d_mode, s_dist, s_fog, s_pat, s_elong, s_eye, out
]))